In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

#This matplotlib module lets you do animation, which we'll be 
#   working with at the end of this exercise
from matplotlib import animation

#tqdm is a simple way to make progress bars. Feel free to delete
#   this if you run into trouble with it, but generating the 
#   animation at the end can take a while, so this can give you
#   a sense of how it's progressing, and if you want to change some
#   settings to make it quicker
from tqdm.auto import tqdm

# Computational Exercise 5: Method of Relaxation

Relaxation is an iterative method that can be used to analytically solve a boundary-value problem such as Laplace's equation. As you may remember from Griffiths (look at page 114-119), the two main properties of the Laplace equation are that V(x) is always the average of the potential of the points around it, and that Laplace's equation tolerates no local extrema. 

The method of relaxation relies primarily on the fact that each point is the average of the points around it. To use this method you:
1. Define the boundary values of V along the boundary
2. Provide reasonable guesses for V on a grid of interior points
3. Iterate through, making the value of each point equal to the average of the points around it
4. Repeat step 3 until the numbers begin to settle

After a few iterations, more passes will produce little change, and we'll have found a numerical solution to Laplace's equation.

## Graded work

**For turning this assignment in, run it twice with the two different sets of boundary conditions listed be and copy your graphs of the solutions (step 3) to the homework turn-in folder.** 

### a) First, the setup in Griffiths Ch. 3, Ex. 4
> Two infinitely-long grounded metal plates at $y = 0$ and $y = a$, are connected at $x = \pm b$ by metal strips maintained at constant potential $V_0$, as shown in Fig. 3.20 (a thin layer of insulation at each corner prevents them from shorting out). Find the potential inside the resulting rectangular pipe.

Take $a=1$, $b=1$, and $V_0 = 2$.

This first setup should prove to you the effectiveness of this method, since you can check it analytically (and by looking at the graph in Griffiths). 

### b) A more complex case
Boundary conditions:
$$x(0) = \sinh(y)$$
$$x(1) = e^y$$
$$y(0) = x$$
$$y(1) = 0$$

Include your graph of your solutions below (and make sure you don't over-write your answer to part a!).

Note that this second setup is one that can't be solved as easily as the first one, and thus shows the power of this numerical method.

## Step 1: Defining the boundary values

First, we'll need to generate our meshgrids, potential array, and boundary values. The next cell starts out with the arrays initialized for you. The <code>*</code> operator allows you to unpack the values of a tuple or array. I've used it to make plotting the grid a little easier down the road, since I can just feed these variables into <code>ax.set_xlim</code> and <code>ax.set_ylim</code>.

I'd highly recommend trying out some different boundary values in order to test your code. I found that setting all of my boundaries to 1 made it easy to see if the algorithm was working correctly. 

In [ ]:
# The resolution of the grid on a side
#   I found that 50 was a good midpoint for both high resolution
#   and speed. Remember that the time it will take to run goes up
#   as the square of this, so if you double it that will quadruple
#   the run time!
res = 50 

# Edit these for different conditions,
a = 1 # Griffiths, Ch 3, Ex 4
b = 1
v0 = 2
xlim = (-b, b)
ylim = (0, a)

# Define the grid of locations V will be solved for
x = np.linspace(*xlim, res)
y = np.linspace(*ylim, res)

X, Y = np.meshgrid(x, y)

V = np.zeros((res, res))

### SET BOUNDARY VALUES HERE

# Set the boundary values by changing the value of V along the
# the edges

#/

# First boundaries
V[0, :] = 0
V[-1, :] = 0
V[:, 0] = v0
V[:, -1] = v0

# Second boundaries
xlim = (0, 1)
ylim = (0, 1)
x = np.linspace(*xlim, res)
y = np.linspace(*ylim, res)

X, Y = np.meshgrid(x, y)

V = np.zeros((res, res))

V[0, :] = x
V[-1, :] = np.zeros(res)
V[:, 0] = np.sinh(y)
V[:, -1] = np.e**y
#/

fig = plt.figure(figsize=plt.figaspect(1)*4)  # Square figure
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, V)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$V$")
plt.show()


## Step 2: The Initial Guess

Next, we need to make a good initial guess for the grid. It doesn't need to be perfect (looking up the analytical answer to this would defeat the purpose of the exercise), but it should be something that makes sense to you, intuitively. Maybe you want to do some sort of trig function, a conic section, or a linear function. The goal here is something close enough. If you choose poorly it'll take a long, long time for your code to run. If you choose really poorly (like making the values huge compared to $V_0$), it's possible you could get the code to fail.

To calculate, you'll probably want to iterate through the points of the V array, skipping the edge rows. Remember that you can use the indices of the meshgrids to find the cartesian values for calculating. For example, <code>X[2, 3]</code> will give you the value of the x-coordinate at index [2, 3].

A convenient way to set up your for loops is with <code>range(start, stop, step)</code>. This function returns a sequence of numbers, where start and step are optional. If you want to skip the first and last rows you can use <code>for i in range(1, res - 1):</code>.

In [ ]:
### Put your inital guess code here!
### You'll be changing values in V, but you'll want to 
### save a copy of the initial value of V
### so that you can comare against it later.

#/
# First boundaries
for i in range(1, res - 1):
    for j in range(1, res - 1):
        V[i, j] = (-np.cos(X[i,j]*np.pi) + np.sin(Y[i,j]*np.pi))/1.5 + 2/3

# Second boundaries
for i in range(1, res - 1):
    for j in range(1, res - 1):
        V[i, j] = 1
#/
        
init_guess = V.copy() # Make a copy of V showing the initial guess

fig = plt.figure(figsize=plt.figaspect(1)*4)  # Square figure
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, V)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$V$")
plt.show()


## Step 3: Sit Back and Relax

For the last section, we'll need to iterate through each point on V, excluding the edges, and take the mean of the adjacent points. Whether or not you want to include the diagonal points is up to you (I didn't). If you do go with the nearest 8 points rather than 4, remember to use a weighted average in order to account for the diagonal points being further away.

While iterating through is easy enough, using a <code>for</code> loop in this doesn't really make sense, since we want to keep going until the difference between successive iterations gets very small. Instead we'll use a <code>while</code> loop (https://www.w3schools.com/python/python_while_loops.asp), which keeps on iterating until a condition is met. 

A good way to determine the difference between successive iterations is to look at the Euclidean (or $L^2$) norm. This is defined as
$$\lvert x \rvert = \sqrt{\sum_{i=0, j=0}^{k} \lvert p_{i, j}^{k+1} - p_{i, j}^{k} \rvert ^2}$$

Where $p_{i, j}^k$ is point $(i, j)$ in the $k^{th}$ iteration of the array, Using this difference, divided by the Euclidean norm of the previous array, will give us the percent change. This full equation would look like:

$$\lvert x \rvert = \frac{\sqrt{\sum_{i=0, j=0}^{k} \lvert p_{i, j}^{k+1} - p_{i, j}^{k} \rvert ^2}}{\sqrt{\sum_{i=0, j=0}^{k} \lvert p_{i, j}^{k} \rvert ^2}}$$

This seems a little complicated, but we can do it simply, using the calculation:
<pre><code>L2_error = np.sqrt(np.sum((V - V_last)**2)/np.sum(V_last**2))
</pre></code>


I found that a good target for the change in the $L^2$ norm was $10^{-5}$. At the beginning of each loop, make a variable equal to a copy of the current V (you can do this with <code>V_last = V.copy()</code>). Since V is an array object, setting a variable to it (for example, <code>V_last = V</code>) won't save the old version (it'll make a new reference to the same instance so when $V$ changes, so will $V_last$).

Also, as you're looping through, set it up to count the number of loops you go through and print that. Name the variable  for this <code>total_iter</code> to match the given code later.  This number will be useful in part 4.

In [ ]:
#/
def mean(i, j, p):
    return (p[i-1, j] + p[i, j-1] + p[i+1, j] + p[i, j+1])/4

from time import perf_counter
m = 0
total_iter = 0
a = perf_counter()

L2_error = 1
L2_target = 1e-5
while L2_error > L2_target:
    V_last = V.copy()
    for i in range(1, res-1):
        for j in range(1, res-1):
            V[i, j] = mean(i, j, V_last)
            m += 1
    L2_error = np.sqrt(np.sum((V - V_last)**2)/np.sum(V_last**2))
    total_iter += 1
    
b = perf_counter()-a            
print("This ran {} calculations in {:.1f} seconds and {} loops".format(m, b, total_iter))
#/

#Feel free to edit this to make it look how you'd like
fig = plt.figure(figsize=plt.figaspect(1)*4)  # Square figure
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, V)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$V$")
ax.set_zlim(0, v0)
plt.show()
fig.savefig('CE5_solution.png') #Change the filename here to whatever you would like


## Part 4: Movie Time

One more thing that might be useful is to animate this process so we can see how it works. You've already written the hard parts of the code by now, so this will be more of a walkthrough of how to turn it into an animated movie.

The bones of the code are laid out for you, so you should be able to finish this section mostly by copy-and-pasting your old code and some minor edits (marked by the all-caps comments).

This code uses the <code>animation.FuncAnimation</code> method, which works by running a function for each frame. This function rewrites the Axis object each time, and <code>FuncAnimation</code> runs for the specified number of frames.

At the end, it'll have saved an html file to the same directory as this file, and that can be opened in your browser to see a nice animation of the method in action! **This file does not need to be included when you turn in this exercise.**

In [ ]:
# os.getcwd lets us find the final file location
from os import getcwd

# First set up the figure and axes
fig = plt.figure(figsize=plt.figaspect(1)*4)  # Square figure
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_zlim(0, v0)

# Set V back to initial conditions
V = init_guess.copy()

# This will be the number of iterations in between frames
frame_skip = 13

# Set total length of progress bar to the number of frames + 2
#   The 2 comes from the first and last frame
pbar = tqdm(total=int(total_iter/frame_skip) + 2)

#Initialize the surface
surface = None

def animate(frames):
    global surface, pbar
    
    # Remove the wframe from the axis
    if surface:
        ax.collections.remove(surface)
    
    ###PLOT THE WIREFRAME/SURFACE
    surface = ax.plot_wireframe(X, Y, V)
        
    for n in range(frame_skip):
    ### PUT YOUR CODE FOR EACH ITERATION HERE (THE STUFF INSIDE
    ###    THE WHILE LOOP)
    ###    Make sure to leave out the part where you update 
    ###    total_iter, as that will cause an error
    #/
        for i in range(1, res-1):
            for j in range(1, res-1):
                V[i, j] = mean(i, j, V)
    #/
    
    #Update the progress bar by 1
    pbar.update(1)


# This makes the actual animation. Total_iter is the count of loops
#   it took me to get to the final relaxation. The frames keyword 
#   tells the function how many total frames there should be. By
#   setting it up like this, we get it to end at the same place
#   as the original code while skipping enough iterations to make
#   the file size reasonable and the movie fairly short.
ani = animation.FuncAnimation(fig, animate,
                              frames = int(total_iter/frame_skip))

ani.save("wireframe_anim.html", writer='html')
print("The animation is saved at {}/wireframe_anim.html".format(getcwd()))